# Introduction
This competition is about identifying steel defects by the means of *semantic segmentation*. 

In this notebook we are going to perform an Exploratory Data Analysis (EDA) and train a ResUnet model in TensorFlow.

Learning goals:

- Understand run length encoding
- Learn how to create a data loader for the segmentation dataset
- Learn how to build a Unet in Tensorflow

In [ ]:
import numpy as np
import pandas as pd

import os
from os.path import join

from collections import defaultdict

import matplotlib.pyplot as plt
import seaborn as sns

import cv2

from tqdm import tqdm


# Main Configuration

In [ ]:
DATA_DIR = join('..', 'input')

TRAIN_IMG_DIR = join(DATA_DIR, 'train_images')
TEST_IMG_DIR = join(DATA_DIR, 'test_images')
SAMPLE_SUB = join(DATA_DIR, 'sample_submission.csv')
TRAIN_DATA = join(DATA_DIR, 'train.csv')
model_save_path = join('.', 'ResUNetSteel_z.h5')
pretrained_model_path = join('..', 'input', 'severstal-pretrained-model', 'ResUNetSteel_z.h5')


train_df = pd.read_csv(TRAIN_DATA)
train_df['ImageId_ClassId'] = train_df.apply(lambda x: '{}_{}'.format(x.ImageId, x.ClassId), axis=1)
sub_df = pd.read_csv(SAMPLE_SUB)
save_model = True


# Kernel Configurations
make_submission = False # used to turn off lengthy model analysis so a submission version doesn't run into memory error
load_pretrained_model = True # load a pre-trained model


## Run-Length Encoding
> In order to reduce the submission file size, our metric uses run-length encoding on the pixel values. Instead of submitting an exhaustive list of indices for your segmentation, you will submit **pairs of values** that contain a **start position and a run length**. E.g. '1 3' implies starting at pixel 1 and running a total of 3 pixels (1,2,3).
>
>The competition format requires a **space delimited list of pairs**. For example, '1 3 10 5' implies pixels 1,2,3,10,11,12,13,14 are to be included in the mask. The metric checks that the pairs are **sorted, positive, and the decoded pixel values are not duplicated**. The pixels are numbered from top to bottom, then left to right: 1 is pixel (1,1), 2 is pixel (2,1), etc.

So, if we were to encode something like our example above, we would have to write it as follows:

In [ ]:
# a more elaborate version of kaggle.com/paulorzp/rle-functions-run-lenght-encode-decode
# note that we will transpose the incoming array outside of the function, 
# as I find this a clearer illustration

def mask_to_rle(mask):
    """
    params:  mask - numpy array
    returns: run-length encoding string (pairs of start & length of encoding)
    """
    
    # turn a n-dimensional array into a 1-dimensional series of pixels
    # for example:
    #     [[1. 1. 0.]
    #      [0. 0. 0.]   --> [1. 1. 0. 0. 0. 0. 1. 0. 0.]
    #      [1. 0. 0.]]
    flat = mask.flatten()
    
    # we find consecutive sequences by overlaying the mask
    # on a version of itself that is displaced by 1 pixel
    # for that, we add some padding before slicing
    padded = np.concatenate([[0], flat, [0]])
    
    # this returns the indices where the sliced arrays differ
    runs = np.where(padded[1:] != padded[:-1])[0] 
    # indexes start at 0, pixel numbers start at 1
    runs += 1

    # every uneven element represents the start of a new sequence
    # every even element is where the run comes to a stop
    # subtract the former from the latter to get the length of the run
    runs[1::2] -= runs[0::2]
 
    # convert the array to a string
    return ' '.join(str(x) for x in runs)

In [ ]:
def rle_to_mask(lre, shape=(1600, 256)):
    '''
    params:  rle   - run-length encoding string (pairs of start & length of encoding)
             shape - (width,height) of numpy array to return 
    
    returns: numpy array with dimensions of shape parameter
    '''    
    # the incoming string is space-delimited
    runs = np.asarray([int(run) for run in lre.split(' ')])
    
    # we do the same operation with the even and uneven elements, but this time with addition
    runs[1::2] += runs[0::2]
    # pixel numbers start at 1, indexes start at 0
    runs -= 1
    
    # extract the starting and ending indeces at even and uneven intervals, respectively
    run_starts, run_ends = runs[0::2], runs[1::2]
    
    # build the mask
    h, w = shape
    mask = np.zeros(h*w, dtype=np.uint8)
    for start, end in zip(run_starts, run_ends):
        mask[start:end] = 1
    
    # transform the numpy array from flat to the original image shape
    return mask.reshape(shape).T

<a id="1"></a> <br>

## Prediction Output Format
The following explanation is taken from here: https://www.kaggle.com/robinteuwens/mask-rcnn-detailed-starter-code

From the competition's [data](https://www.kaggle.com/c/severstal-steel-defect-detection/data) page:
> Each image may have no defects, a defect of a single class, or defects of multiple classes. For each image you must segment defects of each class ```(ClassId = [1, 2, 3, 4])```.

The submission format requires us to make the classifications for each respective class on a separate row, adding the *_class* to the imageId:
![format](https://i.imgur.com/uEeoOQg.png)

## Loss Function

### Dice Coefficient
From the [evaluation](https://www.kaggle.com/c/severstal-steel-defect-detection/overview/evaluation) page:

> This competition is evaluated on the mean Dice coefficient. The Dice coefficient can be used to compare the pixel-wise agreement between a predicted segmentation and its corresponding ground truth. The formula is given by:
>
>$$Dice(X,Y) = \frac{2∗|X∩Y|}{|X|+|Y|}$$
>
>
>where X is the predicted set of pixels and Y is the ground truth. The Dice coefficient is defined to be 1 when both X and Y are empty. The leaderboard score is the mean of the Dice coefficients for each ```<ImageId, ClassId>``` pair in the test set.

Visual illustration of the Dice Coefficient:
![dice_viz](https://i.imgur.com/zl2W0xQ.png)



# Modeling

In [ ]:
import tensorflow as tf
import tensorflow as tf
import tensorflow.keras as keras
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from tensorflow.keras import layers

from sklearn.model_selection import train_test_split

In [ ]:
# network configuration parameters
# original image is 1600x256, so we will resize it
img_w = 512 # resized weidth
img_h = 128 # resized height
batch_size = 8
epochs = 20
# batch size for training unet
val_size = .20 # split of training set between train and validation set
# we will repeat the images with lower samples to make the training process more fair
repeat = False
# only valid if repeat is True
class_1_repeat = 1 # repeat class 1 examples x times
class_2_repeat = 1
class_3_repeat = 1
class_4_repeat = 1

## Data Generator

In [ ]:
class DataGenerator(tf.keras.utils.Sequence):
    def __init__(self, list_ids, labels, image_dir, batch_size=32,
                 img_h=256, img_w=512, shuffle=True):
        
        self.list_ids = list_ids
        self.labels = labels
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.img_h = img_h
        self.img_w = img_w
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        'denotes the number of batches per epoch'
        return int(np.floor(len(self.list_ids)) / self.batch_size)
    
    def __getitem__(self, index):
        'generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # get list of IDs
        list_ids_temp = [self.list_ids[k] for k in indexes]
        # generate data
        X, y = self.__data_generation(list_ids_temp)
        # return data 
        return X, y
    
    def on_epoch_end(self):
        'update ended after each epoch'
        self.indexes = np.arange(len(self.list_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_ids_temp):
        'generate data containing batch_size samples'
        X = np.empty((self.batch_size, self.img_h, self.img_w, 1))
        y = np.empty((self.batch_size, self.img_h, self.img_w, 4))
        
        for idx, id in enumerate(list_ids_temp):
            file_path =  os.path.join(self.image_dir, id)
            image = cv2.imread(file_path, 0)
            image_resized = cv2.resize(image, (self.img_w, self.img_h))
            image_resized = np.array(image_resized, dtype=np.float64)
            # standardization of the image
#             image_resized -= image_resized.mean()
#             image_resized /= image_resized.std()
            image_resized = image_resized.astype('float') / 255.
            mask = np.empty((img_h, img_w, 4))
            
            for idm, image_class in enumerate(['1','2','3','4']):
                rle = self.labels.get(id + '_' + image_class)
                # if there is no mask create empty mask
                if rle is None:
                    class_mask = np.zeros((1600, 256))
                else:
                    class_mask = rle_to_mask(rle, (1600, 256))
             
                class_mask_resized = cv2.resize(class_mask, (self.img_w, self.img_h))
                mask[..., idm] = class_mask_resized
            
            X[idx,] = np.expand_dims(image_resized, axis=2)
            y[idx,] = mask
        
        # normalize Y
        y = (y > 0).astype(int)
            
        return X, y

In [ ]:
# create a dict of all the masks
masks = {}
for index, row in train_df.iterrows():
    masks[row['ImageId_ClassId']] = row['EncodedPixels']

In [ ]:
# repeat low represented samples more frequently to balance our dataset
if repeat:
    class_1_img_id = train_df[(train_df['EncodedPixels']!=-1) & (train_df['ClassId']=='1')]['ImageId'].values
    class_1_img_id = np.repeat(class_1_img_id, class_1_repeat)
    class_2_img_id = train_df[(train_df['EncodedPixels']!=-1) & (train_df['ClassId']=='2')]['ImageId'].values
    class_2_img_id = np.repeat(class_2_img_id, class_2_repeat)
    class_3_img_id = train_df[(train_df['EncodedPixels']!=-1) & (train_df['ClassId']=='3')]['ImageId'].values
    class_3_img_id = np.repeat(class_3_img_id, class_3_repeat)
    class_4_img_id = train_df[(train_df['EncodedPixels']!=-1) & (train_df['ClassId']=='4')]['ImageId'].values
    class_4_img_id = np.repeat(class_4_img_id, class_4_repeat)
    train_image_ids = np.concatenate([class_1_img_id, class_2_img_id, class_3_img_id, class_4_img_id])
else:
    # split the training data into train and validation set (stratified)
    train_image_ids = train_df['ImageId'].unique()

In [ ]:
X_train, X_val = train_test_split(train_image_ids, test_size=val_size, random_state=42)

In [ ]:
params = {'img_h': img_h,
          'img_w': img_w,
          'image_dir': TRAIN_IMG_DIR,
          'batch_size': batch_size,
          'shuffle': True}

# Get Generators
training_generator = DataGenerator(X_train, masks, **params)
validation_generator = DataGenerator(X_val, masks, **params)

In [ ]:
# check out the shapes
x, y = training_generator.__getitem__(1)
print(x.shape, y.shape)

In [ ]:
# visualize steel image with four classes of faults in seperate columns
def viz_steel_img_mask(img, masks):
    img = cv2.cvtColor(img.astype('float32'), cv2.COLOR_BGR2RGB)
    fig, ax = plt.subplots(nrows=1, ncols=4, sharey=True, figsize=(20,10))
    cmaps = ["Oranges", "Blues", "Purples", "Reds"]
    for idx, mask in enumerate(masks):
        ax[idx].imshow(img)
        ax[idx].imshow(mask, alpha=0.3, cmap=cmaps[idx])

In [ ]:
# lets visualize some images with their faults to make sure our data generator is working like it should
x, y = training_generator[np.random.randint(len(training_generator))]
for ix in range(0,batch_size):
    if y[ix].sum() > 0:
        img = x[ix]
        masks_temp = [y[ix][...,i] for i in range(0,4)]
        viz_steel_img_mask(img, masks_temp)

## Building Model

For this competition we are going to use ResUnet [[paper]](https://arxiv.org/pdf/1904.00592.pdf). 

<img src='https://www.researchgate.net/profile/Diab_Abueidda2/publication/344100424/figure/fig2/AS:932383985528836@1599309073617/Illustration-of-the-a-architecture-of-the-ResUNet-b-building-block-used-in-U-Net-and.jpg'>
<center> Illustration of the a) architecture of the ResUNet, b) building block used in U-Net, and c) building block used in ResUNet. </center>

In [ ]:
# Source:
# https://github.com/fchollet/deep-learning-models/blob/master/resnet50.py


pretrained_url = (
    "https://github.com/fchollet/deep-learning-models/"
    "releases/download/v0.2/"
    "resnet50_weights_tf_dim_ordering_tf_kernels_notop.h5"
)


def one_side_pad(x):
    x = ZeroPadding2D((1, 1))(x)
    x = Lambda(lambda x: x[:, :-1, :-1, :])(x)
    return x


def identity_block(input_tensor, kernel_size, filters, stage, block):
    """The identity block is the block that has no conv layer at shortcut.
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    """
    filters1, filters2, filters3 = filters

    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"

    x = Conv2D(filters1, (1, 1), name=conv_name_base + "2a")(input_tensor)
    x = BatchNormalization(name=bn_name_base + "2a")(x)
    x = Activation("relu")(x)

    x = Conv2D(filters2, kernel_size, padding="same", name=conv_name_base + "2b")(x)
    x = BatchNormalization(name=bn_name_base + "2b")(x)
    x = Activation("relu")(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + "2c")(x)
    x = BatchNormalization(name=bn_name_base + "2c")(x)

    x = layers.add([x, input_tensor])
    x = Activation("relu")(x)
    return x


def conv_block(input_tensor, kernel_size, filters, stage, block, strides=(2, 2)):
    """conv_block is the block that has a conv layer at shortcut
    # Arguments
        input_tensor: input tensor
        kernel_size: defualt 3, the kernel size of middle conv layer at
                     main path
        filters: list of integers, the filterss of 3 conv layer at main path
        stage: integer, current stage label, used for generating layer names
        block: 'a','b'..., current block label, used for generating layer names
    # Returns
        Output tensor for the block.
    Note that from stage 3, the first conv layer at main path is with
    strides=(2,2) and the shortcut should have strides=(2,2) as well
    """
    filters1, filters2, filters3 = filters

    conv_name_base = "res" + str(stage) + block + "_branch"
    bn_name_base = "bn" + str(stage) + block + "_branch"

    x = Conv2D(filters1, (1, 1), strides=strides, name=conv_name_base + "2a")(
        input_tensor
    )
    x = BatchNormalization(name=bn_name_base + "2a")(x)
    x = Activation("relu")(x)

    x = Conv2D(filters2, kernel_size, padding="same", name=conv_name_base + "2b")(x)
    x = BatchNormalization(name=bn_name_base + "2b")(x)
    x = Activation("relu")(x)

    x = Conv2D(filters3, (1, 1), name=conv_name_base + "2c")(x)
    x = BatchNormalization(name=bn_name_base + "2c")(x)

    shortcut = Conv2D(filters3, (1, 1), strides=strides, name=conv_name_base + "1")(
        input_tensor
    )
    shortcut = BatchNormalization(name=bn_name_base + "1")(shortcut)

    x = layers.add([x, shortcut])
    x = Activation("relu")(x)
    return x


def resnet50_encoder(input_shape, weights="imagenet"):
    assert input_shape[0] % 32 == 0
    assert input_shape[1] % 32 == 0

    img_input = Input(shape=input_shape)

    x = ZeroPadding2D((3, 3))(img_input)
    x = Conv2D(64, (7, 7), strides=(1, 1), name="conv1")(x)
    f1 = x

    x = BatchNormalization(name="bn_conv1")(x)
    x = Activation("relu")(x)
    x = MaxPooling2D((3, 3), strides=(2, 2))(x)

    x = conv_block(x, 3, [64, 64, 256], stage=2, block="a", strides=(1, 1))
    x = identity_block(x, 3, [64, 64, 256], stage=2, block="b")
    x = identity_block(x, 3, [64, 64, 256], stage=2, block="c")
    f2 = one_side_pad(x)

    x = conv_block(x, 3, [128, 128, 512], stage=3, block="a")
    x = identity_block(x, 3, [128, 128, 512], stage=3, block="b")
    x = identity_block(x, 3, [128, 128, 512], stage=3, block="c")
    x = identity_block(x, 3, [128, 128, 512], stage=3, block="d")
    f3 = x

    x = conv_block(x, 3, [256, 256, 1024], stage=4, block="a")
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block="b")
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block="c")
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block="d")
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block="e")
    x = identity_block(x, 3, [256, 256, 1024], stage=4, block="f")
    f4 = x

    x = conv_block(x, 3, [512, 512, 2048], stage=5, block="a")
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block="b")
    x = identity_block(x, 3, [512, 512, 2048], stage=5, block="c")
    f5 = x

    x = AveragePooling2D((7, 7), name="avg_pool")(x)
    # f6 = x

    if weights == "imagenet":
        weights_path = keras.utils.get_file(
            pretrained_url.split("/")[-1], pretrained_url
        )
        Model(img_input, x).load_weights(weights_path)

    return img_input, [f1, f2, f3, f4, f5]


def unet(input_shape, n_classes, encoder, l1_skip_conn=True, weights='imagenet', output_act_f='softmax'):

    img_input, levels = encoder(input_shape=input_shape, weights=weights)
    [f1, f2, f3, f4, f5] = levels
    

#     x = f5
#     x = ZeroPadding2D((1, 1))(x)
#     x = Conv2D(512, (3, 3), padding="valid", activation="relu")(x)
#     x = BatchNormalization()(x)
#     x = UpSampling2D((2, 2))(x)
#     x = Concatenate(axis=-1)([x, f4])
    
    x = f4
    x = ZeroPadding2D((1, 1))(x)
    x = Conv2D(512, (3, 3), padding="valid", activation="relu")(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Concatenate(axis=-1)([x, f3])
    
    x = ZeroPadding2D((1, 1))(x)
    x = Conv2D(256, (3, 3), padding="valid", activation="relu")(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Concatenate(axis=-1)([x, f2])
    
    x = ZeroPadding2D((1, 1))(x)
    x = Conv2D(128, (3, 3), padding="valid", activation="relu")(x)
    x = BatchNormalization()(x)
    x = UpSampling2D((2, 2))(x)
    x = Concatenate(axis=-1)([x, f1])

    x = ZeroPadding2D((1, 1))(x)
    x = Conv2D(64, (3, 3), padding="valid", activation="relu")(x)
    x = BatchNormalization()(x)

    x = Conv2D(n_classes, (3, 3), padding="same", activation=output_act_f)(x)

    model = Model(inputs=img_input, outputs=x)

    return model

### Loss and Performance Metric

In [ ]:
# Dice similarity coefficient loss, brought to you by: https://github.com/nabsabraham/focal-tversky-unet
def dsc(y_true, y_pred):
    smooth = 1.
    y_true_f = Flatten()(y_true)
    y_pred_f = Flatten()(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    score = (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)
    return score

def dice_loss(y_true, y_pred):
    loss = 1 - dsc(y_true, y_pred)
    return loss

def bce_dice_loss(y_true, y_pred):
    loss = binary_crossentropy(y_true, y_pred) + dice_loss(y_true, y_pred)
    return loss

# Focal Tversky loss, brought to you by:  https://github.com/nabsabraham/focal-tversky-unet
def tversky(y_true, y_pred, smooth=1e-6):
    y_true_pos = tf.keras.layers.Flatten()(y_true)
    y_pred_pos = tf.keras.layers.Flatten()(y_pred)
    true_pos = tf.reduce_sum(y_true_pos * y_pred_pos)
    false_neg = tf.reduce_sum(y_true_pos * (1-y_pred_pos))
    false_pos = tf.reduce_sum((1-y_true_pos)*y_pred_pos)
    alpha = 0.7
    return (true_pos + smooth)/(true_pos + alpha*false_neg + (1-alpha)*false_pos + smooth)

def tversky_loss(y_true, y_pred):
    return 1 - tversky(y_true, y_pred)

def focal_tversky_loss(y_true,y_pred):
    pt_1 = tversky(y_true, y_pred)
    gamma = 0.75
    return tf.keras.backend.pow((1-pt_1), gamma)

In [ ]:
model = unet(input_shape=(img_h, img_w, 1), encoder=resnet50_encoder, n_classes=4, weights=None, output_act_f='sigmoid')


from tensorflow.keras.metrics import binary_accuracy
from tensorflow.keras.optimizers import SGD, Adam, Nadam

model.compile(optimizer = 'adam', loss=dice_loss, metrics=['accuracy', dsc, tversky])

In [ ]:
model.summary()

In [ ]:
if load_pretrained_model:
    try:
        model.load_weights(pretrained_model_path)
        print('pre-trained model loaded!')
    except OSError:
        print('You need to run the model and load the trained model')

### Training

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

best_w = ModelCheckpoint('best.h5',
                                monitor='val_dsc',
                                verbose=0,
                                save_best_only=True,
                                save_weights_only=True,
                                mode='auto',
                                period=1)

last_w = ModelCheckpoint('last.h5',
                                monitor='val_dsc',
                                verbose=0,
                                save_best_only=False,
                                mode='auto',
                                period=1)

callbacks = [best_w, last_w]

history = model.fit_generator(
    generator=training_generator,
    validation_data=validation_generator,
    callbacks=callbacks,
    epochs=epochs,
    verbose=1)

In [ ]:
if save_model: 
    model.save(model_save_path)

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def visualize_training_process(history):
    """ 
    Visualize loss and accuracy from training history
    
    :param history: A Keras History object
    """
    history_df = pd.DataFrame(history.history)
    epochs = np.arange(1, len(history_df) + 1)
    fig = make_subplots(3, 1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['tversky'], mode='lines+markers', name='Tversky'), row=1, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_tversky'], mode='lines+markers', name='Val Tversky'), row=1, col=1)
    
    fig.append_trace(go.Scatter(x=epochs, y=history_df['dsc'], mode='lines+markers', name='Dice'), row=2, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_dsc'], mode='lines+markers', name='Val Dice'), row=2, col=1)
    
    
    
    fig.append_trace(go.Scatter(x=epochs, y=history_df['loss'], mode='lines+markers', name='Loss Train'), row=3, col=1)
    fig.append_trace(go.Scatter(x=epochs, y=history_df['val_loss'], mode='lines+markers', name='Loss Val'), row=3, col=1)
    
    fig.update_layout( xaxis_title="Epochs", template="plotly_white", height=1000)
    
    return fig
visualize_training_process(history)


In [ ]:
# a function to plot image with mask and image with predicted mask next to each other
def viz_single_fault(img, mask, pred, image_class):
    
    fig, ax = plt.subplots(nrows=1, ncols=2, sharey=True, figsize=(15,5))
    
    cmaps = ["Reds", "Blues", "Greens", "Purples"]
    
    ax[0].imshow(img)
    ax[0].imshow(mask, alpha=0.3, cmap=cmaps[image_class-1])
    ax[0].set_title('Mask - Defect Class %s' % image_class)
    
    ax[1].imshow(img)
    ax[1].imshow(pred, alpha=0.3, cmap=cmaps[image_class-1])
    ax[1].set_title('Predicted Mask - Defect Class %s' % image_class)
    
    plt.show()

In [ ]:
# https://www.jeremyjordan.me/evaluating-image-segmentation-models/
def calculate_iou(target, prediction):
    intersection = np.logical_and(target, prediction)
    union = np.logical_or(target, prediction)
    if np.sum(union) == 0:
        iou_score = 0
    else:
        iou_score = np.sum(intersection) / np.sum(union)
    return iou_score

In [ ]:
if not make_submission:
    # lets loop over the predictions and print 5 of each image cases with defects
    count = 0
    # a list to keep count of the number of plots made per image class
    class_viz_count = [0, 0, 0, 0]
    # to keep the total iou score per image class
    class_iou_score = [0, 0, 0, 0]
    # to keep sum of mask pixels per image class
    class_mask_sum = [0, 0, 0, 0]
    # to keep sum of predicted mask pixels per image class
    class_pred_sum = [0, 0, 0, 0]

    # loop over to all the batches in one epoch 
    for i in range(0, 4):
        # get a batch of image, true mask, and predicted mask
        x, y = validation_generator.__getitem__(i)
        predictions = model.predict(x)

        # loop through x to get all the images in the batch
        for idx, val in enumerate(x):
            # we are only interested if there is a fault. if we are dropping images with no faults before this will become redundant
            if y[idx].sum() > 0: 
                # get an image and convert to make it matplotlib.pyplot friendly
                img = x[idx]
                img = cv2.cvtColor(img.astype('float32'), cv2.COLOR_BGR2RGB)
                # loop over the four ourput layers to create a list of all the masks for this image
                masks_temp = [y[idx][...,i] for i in range(0,4)]
                # loop over the four output layers to create a list of all the predictions for this image
                preds_temp = [predictions[idx][...,i] for i in range(0,4)]
                # turn to binary (prediction) mask 
                preds_temp = [p > .5 for p in preds_temp]

                for i, (mask, pred) in enumerate(zip(masks_temp, preds_temp)):
                    image_class = i + 1
                    class_iou_score[i] += calculate_iou(mask, pred)
                    class_mask_sum[i] += mask.sum()
                    class_pred_sum[i] += pred.sum()
                    if mask.sum() > 0 and class_viz_count[i] < 5:
                        viz_single_fault(img, mask, pred, image_class)
                        class_viz_count[i] += 1

In [ ]:
import os
class TestDataGenerator(tf.keras.utils.Sequence):
    def __init__(self, image_dir, batch_size=32, img_h=256, img_w=512, shuffle=False):
        
        self.list_ids = [f.split('/')[-1] for f in glob.glob(join(image_dir, "*.jpg"), recursive=True)]
        self.image_dir = image_dir
        self.batch_size = batch_size
        self.img_h = img_h
        self.img_w = img_w
        self.shuffle = shuffle
        self.on_epoch_end()
    
    def __len__(self):
        'denotes the number of batches per epoch'
        return int(np.floor(len(self.list_ids)) / self.batch_size)
    
    def __getitem__(self, index):
        'generate one batch of data'
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]
        # get list of IDs
        list_ids_temp = [self.list_ids[k] for k in indexes]
        # generate data
        X, list_ids_temp = self.__data_generation(list_ids_temp)
        # return data 
        return X, list_ids_temp
    
    def on_epoch_end(self):
        'update ended after each epoch'
        self.indexes = np.arange(len(self.list_ids))
        if self.shuffle:
            np.random.shuffle(self.indexes)
            
    def __data_generation(self, list_ids_temp):
        'generate data containing batch_size samples'
        X = np.empty((self.batch_size, self.img_h, self.img_w, 1))
        
        for idx, image_id in enumerate(list_ids_temp):
            file_path =  os.path.join(self.image_dir, image_id)
            image = cv2.imread(file_path, 0)
            image_resized = cv2.resize(image, (self.img_w, self.img_h))
            image_resized = np.array(image_resized, dtype=np.float64)
            # standardization of the image
#             image_resized -= image_resized.mean()
#             image_resized /= image_resized.std()
            image_resized = image_resized.astype('float') / 255.
            X[idx, ...] = image_resized[..., None]
            
        return X, list_ids_temp 

In [ ]:
# this is an awesome little function to remove small spots in our predictions
from skimage import morphology

def remove_small_regions(img, size):
    """Morphologically removes small (less than size) connected regions of 0s or 1s."""
    img = morphology.remove_small_objects(img, size)
    img = morphology.remove_small_holes(img, size)
    return img

# a function to apply all the processing steps necessery to each of the individual masks
def process_pred_mask(pred_mask):
    
    pred_mask = cv2.resize(pred_mask.astype('float32'),(256, 1600))
    pred_mask = (pred_mask > .5).astype(int)
    pred_mask = remove_small_regions(pred_mask, 0.02 * np.prod(512))
    pred_mask = mask_to_rle(pred_mask)
    return pred_mask

In [ ]:
import glob
# get all files using glob
test_files = [f for f in glob.glob(join(TEST_IMG_DIR, "*.jpg"), recursive=True)]

test_generator = TestDataGenerator(image_dir=TEST_IMG_DIR, img_h=img_h, img_w=img_w, batch_size=16)

submission_entries = []

if make_submission:
    for test_batch, image_ids in test_generator:
        pred_batch = model.predict(test_batch)

        for i, img_id in enumerate(image_ids):
            for j in range(0, 4):
                lre_mask = process_pred_mask(pred_batch[i, ..., j])
                submission_entries.append((img_id, lre_mask, j + 1))
                
submission = pd.DataFrame(submission_entries, columns=sub_df.columns)

In [ ]:
submission.head()

In [ ]:
submission.to_csv('submission.csv', index=False)